# Enfield, Andrew - DATA 512, A2: Bias in Data

TBD UPDATE

The assignment is at https://wiki.communitydata.cc/HCDS_(Fall_2017)/Assignments#A2:_Bias_in_data.

TBD remove

This notebook pulls, prepares, and analyzes data about the amount of monthly English Wikipedia traffic from January 1, 2008 through September 30, 2017. For more information about the work and data, refer to the [README](Readme.md).

A few notes:
- Normally I'd prefer to keep the explanation and background that's in the README here in the notebook, so everything's in a single file, but I've split it up this time as that's what the assignment requested. I won't copy/paste because keeping duplicate content in sync is horrible.
- Real reproducibility needs tests for the code. A lot of my implementation below is in functions. I'd normally put these functions in at least one separate file that I import into this notebook, and I'd have tests in an additional file. For this assignment I'll just keep everything in this file, for simplicity, even though it means I can't test the code the way I normally would.

# Prereqs

This code requires the libraries as described below.

In [32]:
# load data
import requests
import json
# import os

# load, prepare, and analyze data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from mpl_toolkits.axes_grid.anchored_artists import AnchoredText # for addtl annotations in charts
#from matplotlib.ticker import FuncFormatter # for custom axis labels
from IPython.core.pylabtools import figsize
import seaborn as sns # for formatting
%matplotlib inline 

In [3]:
sns.set_style("whitegrid")
figsize(14,7)

# Load data

TBD UPDATE

This section loads the data from the two APIs described in the README, producing five separate .json files, one for each API and access combination.

In [30]:
d_wikipedia = pd.read_csv('page_data.csv')
d_wikipedia.shape

(47997, 3)

In [31]:
d_wikipedia[:3]

,country,page,last_edit
0,Abkhazia,Zurab Achba,802551672
1,Abkhazia,Garri Aiba,774499188
2,Abkhazia,Zaur Avidzba,803841397


In [18]:
d_population = pd.read_csv('Population Mid-2015.csv', skiprows=2, thousands=',')
d_population.shape

(210, 6)

In [29]:
d_wikipedia.groupby(['country']).size().sort_values(ascending=False)[:10]

country
France           1858
Australia        1610
Pakistan         1268
China            1261
Mexico           1137
United States    1115
Russia           1109
Iran             1055
Spain            1003
India             993
dtype: int64

In [19]:
d_population[:3]

,Location,Location Type,TimeFrame,Data Type,Data,Footnotes
0,Afghanistan,Country,Mid-2015,Number,32247000,NaN
1,Albania,Country,Mid-2015,Number,2892000,NaN
2,Algeria,Country,Mid-2015,Number,39948000,NaN


## Pull article scores

Docs: https://ores.wikimedia.org/v3/#!/scoring/get_v3_scores_context_revid_model

In [103]:
def get_full_ores_score_json(rev_id):
    """TBD referring to https://ores.wikimedia.org/v3/#!/scoring/get_v3_scores_context_revid_model"""
    endpoint = 'https://ores.wikimedia.org/v3/scores/{project}/{revid}/{model}'
    user_agent = 'https://github.com/aenfield'

    params = {'project' : 'enwiki',
              'model' : 'wp10',
              'revid' : rev_id
              }

    api_call = requests.get(endpoint.format(**params), headers = {'User-Agent':'{}'.format(user_agent)})
    return api_call.json()

def get_ores_prediction_from_score_json(json):
    """Return the most likely article type, per ORES. Assumes a JSON dict from Ores with only a single article."""
    return json['enwiki']['scores'][list(j['enwiki']['scores'].keys())[0]]['wp10']['score']['prediction']

def get_ores_prediction(rev_id):
    j = get_full_ores_score_json(rev_id)
    return get_ores_prediction_from_score_json(j)

In [97]:
type(j)

dict

In [104]:
get_ores_prediction('797882120')

'Start'

In [105]:
j = get_full_ores_score_json('797882120')
j

{'enwiki': {'models': {'wp10': {'version': '0.5.0'}},
  'scores': {'797882120': {'wp10': {'score': {'prediction': 'Start',
      'probability': {'B': 0.0325056273665757,
       'C': 0.10161634736900718,
       'FA': 0.003680032854794337,
       'GA': 0.021044772033944954,
       'Start': 0.8081343649161963,
       'Stub': 0.033018855459481376}}}}}}}

In [107]:
list(j['enwiki']['scores'].keys())[0]

'797882120'

In [106]:
get_ores_prediction(j)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [69]:
j['enwiki']['scores']['797882120']['wp10']['score']['prediction']

'Start'

In [84]:
list(j['enwiki']['scores'].keys())[0]

'797882120'

In [85]:
j['enwiki']['scores'][list(j['enwiki']['scores'].keys())[0]]['wp10']['score']['prediction']

'Start'

In [102]:
d_wikipedia['last_edit'][:10].apply(get_ores_prediction)

KeyError: '797882120'